## TP7


* Implementar el detector de fondo naive usando la mediana como
estimador El algoritmo debe recibir el parámetro N (cantidad de
frames utilizados para la estimación) y el intervalo de tiempo para
recalcular el fondo
* Se deben generar las mascaras de foreground y aplicarlas a los frames
para segmentar los objetos en movimiento
* Comparar con alguno de los métodos vistos en la practica basados en
mezcla de gaussiana s


In [2]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib

# Importamos las librerías necesarias
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg
